In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
# fixes weird issue with pyLDAvis (?) warnings

import pandas as pd
import numpy as np
import pickle
import operator
import re
import gc
import gensim
# from gensim.similarities import WmdSimilarity

import pyLDAvis
import pyLDAvis.gensim

# wtf
warnings.filterwarnings("ignore", category=DeprecationWarning)

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from tw_dataset.dbmodels import *
from random import sample
import json

Switching to API Credentials #7


In [2]:
from create_lda_datasets import *

In [2]:
# s = open_session()
# all_tweets_text_es = [t.text for t in s.query(Tweet).all() if t.lang == 'es']

# with open('all_tweets_text_es.json', 'w') as f:
#     json.dump(all_tweets_text_es, f)

In [3]:
f1s = load_nlp_selected_users()

In [5]:
uid, f1 = f1s[4]

In [6]:
uid = int(uid)

In [8]:
from experiments.datasets import load_dataframe

ImportError: No module named graph_tool.all

In [11]:
Xtrain_fname = join(DATAFRAMES_FOLDER, "dfXtrain_%d.pickle" % uid)
Xtest_fname = join(DATAFRAMES_FOLDER, "dfXtestv_%d.pickle" % uid)
Xvalid_fname = join(DATAFRAMES_FOLDER, "dfXvalid_%d.pickle" % uid)

In [12]:
X_train = pd.read_pickle(Xtrain_fname)
X_test = pd.read_pickle(Xtest_fname)
X_valid = pd.read_pickle(Xvalid_fname)

In [3]:
with open('all_tweets_text_es.json') as f:
    all_tweets_text_es = json.load(f)
len(all_tweets_text_es)    

1636480

In [15]:
twids = list(X_train.index) + list(X_valid.index) + list(X_test.index)

In [13]:
s = open_session()

In [16]:
# tweets = sample(all_tweets_text_es, 5000)
tweets = s.query(Tweet.text).filter(Tweet.id.in_(twids)).all()

In [23]:
tweets = [t[0] for t in tweets]

In [17]:
len(tweets)

110687

# OK, let's finally do some topic modelling

In [18]:
from tokenizer import tokenize, spanish_stopwords

In [19]:
def preprocess(doc):
    pre_doc = doc
        
    # remove URLs
    pre_doc = re.sub(
        r"https?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",
        " ", pre_doc)
        
    # find and split hashtags
    # very simple splitting (TODO: come up with something wittier)
    # split on capital letters, but only if hashtag longer than 5
    # → conditional is to avoid splitting abbr. like "IoT" or "NSA"
    pre_doc = re.sub(r"(?:^|\s)[＃#]{1}(\w+)", 
            lambda s: re.sub(r"([A-Z])", r" \1", s.group(0)) if len(s.group(0)) > 5 else s.group(0), 
            pre_doc)
    pre_doc = re.sub(r"＃|#", " ", pre_doc)
    
    # lowercase everything
    pre_doc = pre_doc.lower()
        
    # remove bullshit
    pre_doc = re.sub(r"\@|\'|\"|\\|…|\/|\-|\||\(|\)|\.|\,|\!|\?|\:|\;|“|”|’|—", " ", pre_doc)
    
    # normalize whitespaces
    pre_doc = re.sub(r"\s+", " ", pre_doc)
    pre_doc = re.sub(r"(^\s)|(\s$)", "", pre_doc)
    
    return pre_doc

In [20]:
class get_docs(object):
    def __init__(self, corpus):
        self.corpus = corpus

    def __iter__(self):
        for doc in self.corpus:
            tokens = tokenize(preprocess(doc))
            yield tokens

In [24]:
dictionary = gensim.corpora.Dictionary(get_docs(tweets))
dictionary.filter_extremes(no_below=100, no_above=0.1, keep_n=None)
dictionary.save("tweets_es_%d.dict" % uid)

2017-04-04 21:45:42,245 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-04-04 21:45:53,875 : INFO : adding document #10000 to Dictionary(12310 unique tokens: [u'bocaaabocaa', u'concuerd', u'repasadit', u'spiderm', u'narcotiz']...)
2017-04-04 21:46:05,191 : INFO : adding document #20000 to Dictionary(18459 unique tokens: [u'bocaaabocaa', u'votn', u'concuerd', u'jerarqu', u'repasadit']...)


string index out of range
salí del trabajo me compré un pancho y venía escuchando mi walkman googleen me encararon con una navaja dame la plata ¿gue bgata


2017-04-04 21:46:16,579 : INFO : adding document #30000 to Dictionary(23439 unique tokens: [u'', u'spiderm', u'\xbfdiscut', u'woods', u'\u201cteor']...)
2017-04-04 21:46:27,846 : INFO : adding document #40000 to Dictionary(27438 unique tokens: [u'', u'vittogiann', u'spiderm', u'\xbfdiscut', u'woods']...)
2017-04-04 21:46:38,811 : INFO : adding document #50000 to Dictionary(31052 unique tokens: [u'', u'vittogiann', u'spiderm', u'\xbfdiscut', u'woods']...)
2017-04-04 21:46:49,897 : INFO : adding document #60000 to Dictionary(34321 unique tokens: [u'', u'vittogiann', u'spiderm', u'\xbfdiscut', u'woods']...)
2017-04-04 21:47:00,772 : INFO : adding document #70000 to Dictionary(37417 unique tokens: [u'', u'vittogiann', u'spiderm', u'\xbfdiscut', u'woods']...)
2017-04-04 21:47:11,620 : INFO : adding document #80000 to Dictionary(40100 unique tokens: [u'', u'gauchit', u'vittogiann', u'spiderm', u'\xbfdiscut']...)
2017-04-04 21:47:22,854 : INFO : adding document #90000 to Dictionary(42347 uniq

In [8]:
dictionary = gensim.corpora.Dictionary.load("tweets_es_%d.dict" % uid)

2017-04-03 17:18:28,537 : INFO : loading Dictionary object from tweets_es.dict


In [25]:
bow = [dictionary.doc2bow(doc) for doc in get_docs(tweets)]
with open('tweets_es_bow.pickle', 'wb') as f:
    pickle.dump(bow, f)

string index out of range
salí del trabajo me compré un pancho y venía escuchando mi walkman googleen me encararon con una navaja dame la plata ¿gue bgata


In [9]:
with open('tweets_es_bow.pickle', 'rb') as f:
    bow = pickle.load(f)

In [26]:
n_topics = 10
iters = 100
passes = 10

In [27]:
# gensim.models.ldamodel.LdaModel?

In [28]:
# model = gensim.models.ldamodel.LdaModel(
model = gensim.models.LdaMulticore(
        corpus=bow,
        id2word=dictionary,
        num_topics=n_topics,
        iterations=iters,
        alpha=0.001,
        passes=passes,
        chunksize=10000,
        workers=7
#         distributed=True
)
model.save("tweets_es_u%d_%dtopics.lda" % (uid, n_topics))

2017-04-04 21:57:23,660 : INFO : using serial LDA version on this node
2017-04-04 21:57:23,915 : INFO : running online LDA training, 10 topics, 10 passes over the supplied corpus of 110687 documents, updating every 70000 documents, evaluating every ~110687 documents, iterating 100x with a convergence threshold of 0.001000
2017-04-04 21:57:23,919 : INFO : training LDA model using 7 processes
2017-04-04 21:57:24,046 : INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #10000/110687, outstanding queue size 1
2017-04-04 21:57:24,123 : INFO : PROGRESS: pass 0, dispatched chunk #1 = documents up to #20000/110687, outstanding queue size 2
2017-04-04 21:57:24,200 : INFO : PROGRESS: pass 0, dispatched chunk #2 = documents up to #30000/110687, outstanding queue size 3
2017-04-04 21:57:24,279 : INFO : PROGRESS: pass 0, dispatched chunk #3 = documents up to #40000/110687, outstanding queue size 4
2017-04-04 21:57:24,356 : INFO : PROGRESS: pass 0, dispatched chunk #4 = documents up to #

In [13]:
model = gensim.models.LdaModel.load("tweets_es_u%d_%dtopics.lda" % (uid, n_topics))

2017-04-03 18:42:04,121 : INFO : loading LdaModel object from tweets_es_10topics.lda
2017-04-03 18:42:04,136 : INFO : setting ignored attribute state to None
2017-04-03 18:42:04,137 : INFO : setting ignored attribute dispatcher to None
2017-04-03 18:42:04,137 : INFO : loading LdaModel object from tweets_es_10topics.lda.state


In [30]:
viz = pyLDAvis.gensim.prepare(model, [v for v in bow], model.id2word)
pickle.dump(viz, open("tweets_es_u%d_%d.viz" % (uid, n_topics),'wb'))

In [31]:
viz = pickle.load(open("tweets_es_u%d_%d.viz" % (uid, n_topics),'rb'))
pyLDAvis.display(viz)

# Now we extract LDA feats for small dataset

In [29]:
X_train, X_valid, X_test, y_train, y_valid, y_test = load_small_validation_dataframe(uid)

In [32]:
X_train_inds = X_train.index
X_valid_inds = X_valid.index
X_test_inds = X_test.index

In [33]:
train_tweets = [s.query(Tweet).get(twid) for twid in X_train_inds]
valid_tweets = [s.query(Tweet).get(twid) for twid in X_valid_inds]
test_tweets = [s.query(Tweet).get(twid) for twid in X_test_inds]

In [36]:
def get_lda(t):
    tokens = tokenize(preprocess(t.text))
    doc_bow = dictionary.doc2bow(tokens)
    doc_lda = model[doc_bow]
    
    return doc_lda

In [37]:
rows_train = [get_lda(t) for t in train_tweets]

In [38]:
rows_valid = [get_lda(t) for t in valid_tweets]

In [39]:
rows_test = [get_lda(t) for t in test_tweets]

In [40]:
def rows_to_csc(rows):
    data = []
    row_ind = []
    col_ind = []
    for i, r in enumerate(rows):
        for j, d in r:
            row_ind.append(i)
            col_ind.append(j)
            data.append(d)
    return csc_matrix((data, (row_ind, col_ind)))

X_train_lda = rows_to_csc(list(rows_train))
X_valid_lda = rows_to_csc(list(rows_valid))
X_test_lda = rows_to_csc(list(rows_test))


In [41]:
X_train_combined = sp.hstack((X_train, X_train_lda))
X_valid_combined = sp.hstack((X_valid, X_valid_lda))
X_test_combined = sp.hstack((X_test, X_test_lda))

In [43]:
from experiments._1_one_user_learn_neighbours.classifiers import model_select_svc2

In [44]:
ds_comb = (X_train_combined, X_valid_combined, y_train, y_valid)
comb_clf = model_select_svc2(ds_comb, n_jobs=5)

# Tuning hyper-parameters for f1

Best parameters set found on training set:

{'kernel': 'rbf', 'C': 1, 'gamma': 0.1, 'class_weight': 'balanced'}
Detailed classification report:

Scores on training set.
             precision    recall  f1-score   support

        0.0       1.00      0.80      0.89      3246
        1.0       0.27      0.95      0.42       254

avg / total       0.94      0.81      0.85      3500


Scores on test set.

             precision    recall  f1-score   support

        0.0       0.96      0.77      0.86       462
        1.0       0.19      0.66      0.29        38

avg / total       0.91      0.76      0.81       500




In [45]:
ds = (X_train, X_valid, y_train, y_valid)
clf = model_select_svc2(ds, n_jobs=5)

# Tuning hyper-parameters for f1

Best parameters set found on training set:

{'kernel': 'rbf', 'C': 1, 'gamma': 0.1, 'class_weight': 'balanced'}
Detailed classification report:

Scores on training set.
             precision    recall  f1-score   support

        0.0       0.99      0.80      0.88      3246
        1.0       0.26      0.91      0.40       254

avg / total       0.94      0.80      0.85      3500


Scores on test set.

             precision    recall  f1-score   support

        0.0       0.97      0.77      0.86       462
        1.0       0.20      0.68      0.31        38

avg / total       0.91      0.77      0.82       500


